# lecture3
## 大语言模型应用实战

#### API简单调用

In [ ]:
%pip install openai

以OpenAI为例

但国内目前很难搞到OpenAI的API KEY了，能搞到的同学可以试一试

In [1]:
from openai import OpenAI

client = OpenAI(api_key="sk-1234567890abcdef1234567890abcdef") # key替换成你的API key

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[ # system prompt # user prompt /query
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

APITimeoutError: Request timed out.

以kimi为例

In [2]:
from openai import OpenAI
 
client = OpenAI(
    api_key = "sk-HWm8ZBWVs9DStt3MM1aVWTtelJndUJmoR7rdhaV72Sf9meZG", # key替换成你的API key
    base_url = "https://api.moonshot.cn/v1",
)

completion = client.chat.completions.create(
    model = "moonshot-v1-8k",
    messages = [
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
        {"role": "user", "content": "你好，我叫李雷，1+1等于多少？"}
    ],
    temperature = 0.3, # 控制生成文本的创造性，0.0 为最保守，1.0 为最激进
    max_tokens=1000,
)
 
print(completion.choices[0].message.content)

你好，李雷！1+1 等于 2。这是一个基本的数学加法问题。如果你有其他问题，也可以随时问我。


尝试利用request库调用API，查询API文档，找到对应API的URL，以及需要传递的参数


In [47]:
import requests
import json

url = "https://api.moonshot.cn/v1/chat/completions"
api_key = "sk-HWm8ZBWVs9DStt3MM1aVWTtelJndUJmoR7rdhaV72Sf9meZG" # 替换成你的API key
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# python 的 字典对象 dict()
data = {
    "model": "moonshot-v1-8k",
    "messages": [ 
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"}, 
        {"role": "user","content": "你好，我叫李雷，1+1等于多少？"} 
    ],
}

response = requests.post(url, headers=headers, data=json.dumps(data)) # 把python对象转换成json对象，即json格式的字符串
# response = response.post(url, headers=headers, json=data) # 也可以直接使用json参数，requests库会自动把python对象转换成json对象
print(response)
print(response.json()["choices"][0]["message"]["content"])


<Response [200]>
你好，李雷！1+1 等于 2。这是一个基本的数学加法问题。如果你有其他问题或需要帮助，请随时告诉我。


思考：为啥要转成JSON格式字符串？

### Stream方式调用

In [48]:
from openai import OpenAI
 
client = OpenAI(
    api_key = "sk-HWm8ZBWVs9DStt3MM1aVWTtelJndUJmoR7rdhaV72Sf9meZG", # key替换成你的API key
    base_url = "https://api.moonshot.cn/v1",
)
 
stream = client.chat.completions.create(
    model = "moonshot-v1-8k",
    messages = [
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
        {"role": "user", "content": "你好，我叫李雷，1+1等于多少？"}
    ],
    temperature = 0.3,
    stream=True, # <-- 注意这里，我们通过设置 stream=True 开启流式输出模式
)
 
# 当启用流式输出模式（stream=True），SDK 返回的内容也发生了变化，我们不再直接访问返回值中的 choice
# 而是通过 for 循环逐个访问返回值中每个单独的块（chunk）
 

# 你好，李雷！1+1 等于 2。这是一个基本的数学加法运算。如果你有其他问题或需要帮助，请随时告诉我！


for chunk in stream:
	# 在这里，每个 chunk 的结构都与之前的 completion 相似，但 message 字段被替换成了 delta 字段
	delta = chunk.choices[0].delta # <-- message 字段被替换成了 delta 字段
 
	if delta.content:
		# 我们在打印内容时，由于是流式输出，为了保证句子的连贯性，我们不人为地添加
		# 换行符，因此通过设置 end="" 来取消 print 自带的换行符。
		print(delta.content, end="")

你好，李雷！1+1 等于 2。这是一个基本的数学加法问题。如果你有其他问题或需要帮助，请随时告诉我。

思考：ChatGPT等Web端的打字机效果和我们上面的代码实现的效果有什么区别？

### 一个简单的Chatbot

In [51]:
from openai import OpenAI

api_key = "sk-HWm8ZBWVs9DStt3MM1aVWTtelJndUJmoR7rdhaV72Sf9meZG" # 替换成你的API key
client = OpenAI(api_key=api_key, base_url="https://api.moonshot.cn/v1")

# 初始化消息列表，即系统提示词
messages = [
    {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手。"}
]

#kimi 余额 1000个token 包括我的输入，和模型返回，加起来，超过了1000token


def get_user_input():
    return input("你: ")

def get_ai_response(user_input):
    messages.append({"role": "user", "content": user_input})
    completion = client.chat.completions.create(
        model="moonshot-v1-8k",
        messages=messages,
        temperature=0.3
    )
    print(completion)
    return completion.choices[0].message.content

# 运行聊天循环
print("Kimi: 你好，我是Kimi，你的人工智能助手。")
while True:
    user_input = get_user_input()
    print(f'你: {user_input}')
    if user_input.lower() in ['退出', 'quit', 'exit']:
        print("Kimi: 再见！")
        break
    ai_response = get_ai_response(user_input)
    print(f"Kimi: {ai_response}")

思考：当前能够记忆哪些上下文信息？

### 工具调用

这部分也是完全兼容OpenAI的，即如果你把URL和KEY换成GPT的，下方代码也是可以完成工具调用的

In [ ]:
from typing import * # 引入Dict,Any等类型，用于类型提示，提高代码可读性
 
import json
import requests
from openai import OpenAI
 
client = OpenAI(
    api_key="sk-HWm8ZBWVs9DStt3MM1aVWTtelJndUJmoR7rdhaV72Sf9meZG", # 替换成你的API key
    base_url="https://api.moonshot.cn/v1",
)
  
def search(arguments: Dict[str, Any]) -> Any:
    """搜索使用的是 Tavily 的API
    API文档: https://docs.tavily.com/docs/tavily-api/rest_api
    申请API KEY: https://app.tavily.com/home
    """
    search_api_key="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" # 替换成你的API key
    query = arguments["query"]
    print('searching...')
    data = {
        'api_key': search_api_key,
        'query': query,
    }
    result = result=requests.post('https://api.tavily.com/search', json=data).text
    print(result)
    return {"result": result}
 
 
tool_map = {
    "search": search,
}
 
tools = [
    {
        "type": "function",
        "function":     
        {
            "name": "search",  # 函数的名称，请使用英文大小写字母、数据加上减号和下划线作为函数名称
            "description": """ 
                通过搜索引擎搜索互联网上的内容。
                当你的知识无法回答用户提出的问题，或用户请求你进行联网搜索时，调用此工具。请从与用户的对话中提取用户想要搜索的内容作为 query 参数的值。
                搜索结果包含网站的标题、网站的地址（URL）以及网站简介。
            """,  # 函数的介绍，在这里写上函数的具体作用以及使用场景，以便 Kimi 大模型能正确地选择使用哪些函数
            "parameters": {  # 使用 parameters 字段来定义函数接收的参数
                "type": "object",  # 固定使用 type: object 来使 Kimi 大模型生成一个 JSON Object 参数
                "required": ["query"],  # 使用 required 字段告诉 Kimi 大模型哪些参数是必填项
                "properties": {  # properties 中是具体的参数定义，你可以定义多个参数
                    "query": {  # 在这里，key 是参数名称，value 是参数的具体定义
                        "type": "string",  # 使用 type 定义参数类型
                        "description": """
                            用户搜索的内容，请从用户的提问或聊天上下文中提取。
                        """  # 使用 description 描述参数以便 Kimi 大模型更好地生成参数
                    }
                }
            }
        },
    },
]

 
messages = [
    {"role": "system",
     "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
    {"role": "user", "content": "联网搜索，今天奥运会情况如何?"}  # 在提问中要求 Kimi 大模型联网搜索
]
 
finish_reason = None

while finish_reason is None or finish_reason == "tool_calls":
    completion = client.chat.completions.create(
        model="moonshot-v1-8k",
        messages=messages,
        temperature=0.3,
        tools=tools,  # <-- 我们通过 tools 参数，将定义好的 tools 提交给 Kimi 大模型
    )
    choice = completion.choices[0]
    finish_reason = choice.finish_reason

 
    if finish_reason == "tool_calls":  # <-- 判断当前返回内容是否包含 tool_calls
        messages.append(choice.message)  # <-- 我们将 Kimi 大模型返回给我们的 assistant 消息也添加到上下文中，以便于下次请求时 Kimi 大模型能理解我们的诉求
        for tool_call in choice.message.tool_calls:  # <-- tool_calls 可能是多个，因此我们使用循环逐个执行
            tool_call_name = tool_call.function.name
            tool_call_arguments = json.loads(tool_call.function.arguments)  # <-- arguments 是序列化后的 JSON Object，我们需要使用 json.loads 反序列化一下
            tool_function = tool_map[tool_call_name]  # <-- 通过 tool_map 快速找到需要执行哪个函数
            tool_result = tool_function(tool_call_arguments)
 
            # 使用函数执行结果构造一个 role=tool 的 message，以此来向模型展示工具调用的结果；
            # 注意，我们需要在 message 中提供 tool_call_id 和 name 字段，以便 Kimi 大模型
            # 能正确匹配到对应的 tool_call。
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": tool_call_name,
                "content": json.dumps(tool_result),  # <-- 我们约定使用字符串格式向 Kimi 大模型提交工具调用结果，因此在这里使用 json.dumps 将执行结果序列化成字符串
            })
 
print(choice.message.content)  # <-- 在这里，我们才将模型生成的回复返回给用户


```mermaid
graph TD;
    A(请联网搜索Spark Lab，并告诉我它是什么?)-->AA((LLM))
    AA-->B(模型判断并调用了搜索工具);
    B-->C(搜索工具结果显示, SparkLab is a fun and educational destination for... );
    C--再次调用-->D((LLM))
    D-->F(Spark是一个...)

```